## *Import the Libraries*

In [ ]:
from google.colab import drive
import os
from zipfile import ZipFile 
import shutil
from os.path import exists, join, basename, splitext
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

## *Downloading the Data*

Downloaded the data from the google drive, unzipped the folder and then further unzipped the folder to extract all the videos.

In [ ]:
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/Data/audio-video.zip

In [3]:
for folder in os.listdir('/content/audio-video'):
  with ZipFile('/content/audio-video/' + folder, 'r') as zipObj:
   zipObj.extractall('/content/data/')


In [5]:
shutil.rmtree('/content/audio-video')

In [6]:
os.mkdir('/content/videos/')

In [7]:
for folder in os.listdir('/content/data'):
  for video in os.listdir('/content/data/' + folder):
    shutil.copy('/content/data/' + folder + '/' + video, '/content/videos/' + video)

In [8]:
shutil.rmtree('/content/data')

## *Downloading and installing the OpenFace Library*

Downloaded the OpenFace Library which is a library used for facial emotion detection.

In [ ]:
git_repo_url = 'https://github.com/TadasBaltrusaitis/OpenFace.git'
project_name = splitext(basename(git_repo_url))[0]
# clone openface
!git clone -q --depth 1 $git_repo_url

# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# Get newest GCC
!sudo apt-get update
!sudo apt-get install build-essential 
!sudo apt-get install g++-8

# # install python dependencies
# !pip install -q youtube-dl

# Finally, actually install OpenFace
!cd OpenFace && bash ./download_models.sh && sudo bash ./install.sh

## *Extracting the Action Units (AU)*

Used the OpenFace Library to extract Action Units from the videos of the actors

In [ ]:
!python3 /content/AUsFeatureExtractor.py --videos_dir '/content/videos' --openFace_path /content/OpenFace --out_dir /content/Extracted_AUs --out_dir_processed /content/processed_AUs

## *Preprocessing the Data*

Extracted the Action Units from the videos and store them in a csv file format in order to further process the data. Built a dataframe out of all the AUs and extracted features and labels from it. Further divided the data into train-test split.

In [1]:
def process_AU(emb_path):
  X_total = pd.DataFrame([])
  for video_embs in os.listdir(emb_path):
    embs_df = pd.read_csv(os.path.join(emb_path, video_embs), sep=";")
    aux_df = pd.DataFrame([embs_df.mean()])
    X_total = X_total.append(aux_df)

  return X_total

In [ ]:
X_total = process_AU('/content/processed_AUs')

In [2]:
def clean_df(df):
  df["emotion"] = pd.to_numeric(df["emotion"])
  df = df.drop(["index", "path", "video_name", "actor"], axis=1)
  labels = df["emotion"]
  features = df.drop(["emotion"])
  return features, labels

In [ ]:
features, labels = clean_df(X_total)

In [4]:
def generate_train_test(features, labels):
  X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

  return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = generate_train_test(features, labels)

## *Training the Model*

Used 2 Machine Learning Algorithms SVM and Random Forest Classifier to train on the training dataset of AUs and predicted the result using the test set.

In [ ]:
svm_model = SVC(C = 1.0)
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

In [ ]:
randomForest_model = RandomForestClassifer(n_estimators = 100, max_depth = 3)
randomForest_model.fit(X_train, y_train)

y_pred_rdm = randomForest_model.predict(X_test)

## *Evaluation of Results*

Calculated the Precision, Recall and the F1 score of both the models.

In [ ]:
print("Precision Score for SVM Model is:", precision_score(y_test, y_pred_svm))
print("Recall Score for SVM Model is:", recall_score(y_test, y_pred_svm))
print("F1 Score for SVM Model is:", f1_score(y_test, y_pred_svm))

In [ ]:
print("Precision Score for Random Forest Model is:", precision_score(y_test, y_pred_rdm))
print("Recall Score for Random Forest Model is:", recall_score(y_test, y_pred_rdm))
print("F1 Score for Random Forest Model is:", f1_Score(y_test, y_pred_rdm))